In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('path to intent JSON file').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    if intent['tag'] == "Default":
        classes.append(intent['tag'])
        continue
    for pattern in intent['patterns']:
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

93 documents
18 classes ['angry', 'angry location', 'angry location 1', 'angry location 2', 'bye', 'gratitude', 'greeting', 'happy', 'happy location', 'happy location 1', 'happy location 2', 'music angry', 'music happy', 'music sad', 'sad', 'sad location', 'sad location 1', 'sad location 2']
91 unique lemmatized words ['aku', 'amarah', 'angry', 'bahagia', 'balong', 'banget', 'biar', 'bikin', 'bogor', 'buat', 'buruk', 'bye', 'cai', 'camp', 'dadah', 'damar', 'dong', 'dufan', 'dunia', 'endah', 'fantasi', 'ga', 'gbk', 'good', 'goodbye', 'hai', 'halo', 'happy', 'hari', 'hati', 'hulu', 'ini', 'jakarta', 'jumpa', 'kasih', 'kesal', 'kesedihan', 'kota', 'lagu', 'langit', 'leuwi', 'lokasi', 'makin', 'malam', 'marah', 'membuatku', 'memperbaiki', 'mengatasi', 'menurunkan', 'menyenangkan', 'merasa', 'meredakan', 'merenung', 'mood', 'musik', 'nambah', 'ngerasa', 'pagi', 'patah', 'rasa', 'rekomendasi', 'saat', 'sad', 'sakit', 'sampai', 'sangat', 'saya', 'sea', 'seaworld', 'sedang', 'sedih', 'selamat'

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    bag = [1 if w in pattern_words else 0 for w in words]

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)  # Change the dtype to object
# create train and test lists. X - patterns, Y - intents
train_x = np.array(list(training[:, 0]), dtype=np.float32)
train_y = np.array(list(training[:, 1]), dtype=np.float32)
print("Training data created")

Training data created


In [ ]:
from keras.optimizers import legacy as optimizers

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)




Epoch 1/200
19/19 [==============================] - 1s 3ms/step - loss: 2.8667 - accuracy: 0.1075 
Epoch 2/200
19/19 [==============================] - 0s 3ms/step - loss: 2.7689 - accuracy: 0.1613
Epoch 3/200
19/19 [==============================] - 0s 3ms/step - loss: 2.6546 - accuracy: 0.1398
Epoch 4/200
19/19 [==============================] - 0s 3ms/step - loss: 2.6036 - accuracy: 0.1828
Epoch 5/200
19/19 [==============================] - 0s 4ms/step - loss: 2.4049 - accuracy: 0.2688
Epoch 6/200
19/19 [==============================] - 0s 4ms/step - loss: 2.3329 - accuracy: 0.2473
Epoch 7/200
19/19 [==============================] - 0s 2ms/step - loss: 2.2836 - accuracy: 0.2688
Epoch 8/200
19/19 [==============================] - 0s 2ms/step - loss: 2.1038 - accuracy: 0.3978
Epoch 9/200
19/19 [==============================] - 0s 2ms/step - loss: 2.0625 - accuracy: 0.3548
Epoch 10/200
19/19 [==============================] - 0s 2ms/step - loss: 1.9009 - accuracy: 0.3871
Epoch 11

In [ ]:
model2 = Sequential()
model2.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit the model
history2 = model2.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
19/19 [==============================] - 1s 2ms/step - loss: 2.8773 - accuracy: 0.0753
Epoch 2/200
19/19 [==============================] - 0s 2ms/step - loss: 2.7296 - accuracy: 0.1183
Epoch 3/200
19/19 [==============================] - 0s 2ms/step - loss: 2.5912 - accuracy: 0.1828
Epoch 4/200
19/19 [==============================] - 0s 2ms/step - loss: 2.5535 - accuracy: 0.1290
Epoch 5/200
19/19 [==============================] - 0s 2ms/step - loss: 2.3580 - accuracy: 0.2796
Epoch 6/200
19/19 [==============================] - 0s 3ms/step - loss: 2.2297 - accuracy: 0.3118
Epoch 7/200
19/19 [==============================] - 0s 2ms/step - loss: 2.0854 - accuracy: 0.3871
Epoch 8/200
19/19 [==============================] - 0s 2ms/step - loss: 1.9122 - accuracy: 0.3978
Epoch 9/200
19/19 [==============================] - 0s 2ms/step - loss: 1.8094 - accuracy: 0.4516
Epoch 10/200
19/19 [==============================] - 0s 2ms/step - loss: 1.6512 - accuracy: 0.5376
Epoch 11/

In [ ]:
def clean_up_sentence(sentence):
    # Tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # Lemmatize each word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    # Tokenize and lemmatize the pattern
    sentence_words = clean_up_sentence(sentence)
    # Initialize bag of words
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    # Filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.5
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # Sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = i['responses']
            break
    return random.choice(result)

In [ ]:
# Continuously interact with the model until the input pattern contains the word "bye"
while True:
    # Take user input
    new_pattern = input("You: ")

    # If the input pattern contains the word "bye", exit the loop
    if "bye" in new_pattern.lower():
        print("Bot: Goodbye!")
        break

    # Predict intent and get response
    ints = predict_class(new_pattern, model)
    response = get_response(ints, intents)
    print("Bot:", response)


You: a
1/1 [==============================] - 0s 116ms/step
Bot: Sampai jumpa lagi!
You: kasd
1/1 [==============================] - 0s 24ms/step
Bot: Dadah! Semoga harimu menyenangkan!
